<a href="https://colab.research.google.com/github/1prc/1prc/blob/main/Copy_of_research_banking_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# With IBM Fairness toolkit 360

In [7]:
import sys
import warnings
warnings.filterwarnings('ignore')

!pip install aif360
!pip install 'aif360[LFR,OptimPreproc]'
!pip install 'aif360[all]'
!pip install aif360[Reductions]
# !pip install aif360[datasets]




import numpy as np
from aif360.datasets import GermanDataset, StandardDataset
from aif360.metrics import BinaryLabelDatasetMetric
from aif360.algorithms.preprocessing import Reweighing
from IPython.display import Markdown, display


import pandas as pd

from aif360.datasets import StandardDataset
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from aif360.metrics import ClassificationMetric



In [8]:
# adding created synthetic dataset
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')
df = pd.read_csv('/content/biased_loan_dataset.csv')
print(df.columns)



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Index(['Loan_ID', 'Gender', 'Married', 'Dependents', 'Education',
       'Self_Employed', 'ApplicantIncome', 'CoapplicantIncome', 'LoanAmount',
       'Loan_Amount_Term', 'Credit_History', 'Property_Area', 'Loan_Status'],
      dtype='object')


In [ ]:
print(df["Gender"].unique())

['Male' 'Female']


#Reweighting

In [9]:
# Debug: Check initial dataset shape
print("Initial Dataset Shape:", df.shape)

# Drop Loan_ID if it exists
if "Loan_ID" in df.columns:
    df = df.drop(columns=["Loan_ID"])

# Fill Missing Values Before Encoding
df = df.fillna({
    "Gender": "Unknown",  # Assign 'Unknown' to missing gender values
    "Married": "No",  # Default to 'No' if marital status is missing
    "Dependents": "0",  # Assume no dependents if missing
    "Education": "Graduate",  # Default to 'Graduate'
    "Self_Employed": "No",  # Default to 'No'
    "Credit_History": df["Credit_History"].mode()[0],  # Fill with mode (most common)
    "LoanAmount": df["LoanAmount"].median(),  # Fill missing loan amounts with median
    "Loan_Amount_Term": df["Loan_Amount_Term"].median(),  # Fill with median term
    "Property_Area": "Urban",  # Default to 'Urban'
    "Loan_Status": "N"  # Assume 'N' if Loan Status is missing
})

# Encoding Gender and Loan_Status manually
gender_map = {'Male': 1, 'Female': 0, 'Unknown': -1}  # Male is privileged
loan_status_map = {'Y': 1, 'N': 0}  # 1 means approved

df["Gender"] = df["Gender"].map(gender_map)
df["Loan_Status"] = df["Loan_Status"].map(loan_status_map)

# Debug: Check unique values after encoding
print("Gender Unique Values:", df["Gender"].unique())
print("Loan_Status Unique Values:", df["Loan_Status"].unique())


Initial Dataset Shape: (10000, 13)
Gender Unique Values: [1 0]
Loan_Status Unique Values: [1 0]


In [10]:
# 2. One-hot encode other categorical features
categorical_cols = ['Married', 'Dependents', 'Education', 'Self_Employed', 'Property_Area']
df_encoded = pd.get_dummies(df, columns=categorical_cols, drop_first=True)

# 3. Preserve original order of non-categorical columns
original_numeric_cols = [
     'Gender', 'ApplicantIncome', 'CoapplicantIncome',
    'LoanAmount', 'Loan_Amount_Term', 'Credit_History', 'Loan_Status'
]

# Combine original numeric columns with one-hot encoded columns
df_final = pd.concat([
    df[original_numeric_cols],
    df_encoded.drop(columns=original_numeric_cols)
], axis=1)

# 4. Convert to AIF360 Dataset (drop Loan_ID)
dataset_orig = StandardDataset(
    df_final,
    label_name="Loan_Status",
    favorable_classes=[1],
    protected_attribute_names=["Gender"],
    privileged_classes=[[1]],
    features_to_drop=["Loan_ID"]
)

In [11]:
df.head(10)

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,1,Yes,2,Graduate,Yes,36039,27.147345,10844.231359,12.0,1.0,Urban,1
1,0,Yes,0,Graduate,Yes,24872,1478.290089,7517.009210,120.0,0.0,Urban,0
2,0,Yes,1,Not Graduate,Yes,13766,1509.643045,4178.824845,36.0,1.0,Urban,0
3,1,Yes,2,Graduate,Yes,26321,115.589454,7974.560720,84.0,0.0,Rural,0
4,1,Yes,1,Graduate,Yes,74224,1267.616670,22326.550337,180.0,1.0,Semiurban,1
5,1,No,0,Graduate,Yes,19646,1082.037811,5960.451411,84.0,1.0,Urban,0
6,1,Yes,3+,Not Graduate,Yes,40549,672.076670,12248.711824,360.0,1.0,Urban,0
7,0,No,0,Graduate,No,19742,1290.047766,5927.080979,120.0,1.0,Urban,1
8,1,No,0,Graduate,No,20032,978.113587,6056.272595,360.0,1.0,Rural,1
9,0,No,0,Graduate,No,24791,1756.967998,7476.829525,84.0,1.0,Semiurban,1


In [39]:
# Define print_approval_rates function
def print_approval_rates(dataset, label):
    """Prints the approval rate for privileged and unprivileged groups."""
    privileged_filter = dataset.protected_attributes[:, 0] == 1  # Gender=1 (Male)
    unprivileged_filter = dataset.protected_attributes[:, 0] == 0  # Gender=0 (Female)

    privileged_approval_rate = dataset.labels[privileged_filter].mean()
    unprivileged_approval_rate = dataset.labels[unprivileged_filter].mean()

    print(f"\nApproval Rates - {label}:")
    print(f"  Privileged Group (Male) Approval Rate: {privileged_approval_rate:.4f}")
    print(f"  Unprivileged Group (Female) Approval Rate: {unprivileged_approval_rate:.4f}")

# 3. Split Data
dataset_orig_train, dataset_orig_test = dataset_orig.split([0.7], shuffle=True)

# 4. Train Baseline Model
model = LogisticRegression(solver='liblinear')
model.fit(dataset_orig_train.features, dataset_orig_train.labels.ravel())
y_pred = model.predict(dataset_orig_test.features)

# 5. Fairness Metrics Before Mitigation
dataset_pred = dataset_orig_test.copy()
dataset_pred.labels = y_pred
print_approval_rates(dataset_orig_test, "Original Test Data")

metric_test = ClassificationMetric(
    dataset_orig_test, dataset_pred,
    privileged_groups=[{"Gender": 1}], unprivileged_groups=[{"Gender": 0}]
)

print("\nFairness Metrics Before Mitigation:")
print(f" Accuracy: {accuracy_score(dataset_orig_test.labels, y_pred):.4f}")
print(f" Disparate Impact (DI): {metric_test.disparate_impact():.4f}")
print(f" Statistical Parity Difference (SPD): {metric_test.statistical_parity_difference():.4f}")
print(f" Equal Opportunity Difference (EOD): {metric_test.equal_opportunity_difference():.4f}")
print(f" Average Odds Difference (AOD): {metric_test.average_odds_difference():.4f}")

# 6. Apply Reweighing Mitigation
reweigh = Reweighing(
    privileged_groups=[{"Gender": 1}],
    unprivileged_groups=[{"Gender": 0}]
)
dataset_train_transf = reweigh.fit_transform(dataset_orig_train)

model = LogisticRegression(solver='liblinear')
model.fit(
    dataset_train_transf.features,
    dataset_train_transf.labels.ravel(),
    sample_weight=dataset_train_transf.instance_weights
)
y_pred = model.predict(dataset_orig_test.features)

# 7. Fairness Metrics After Mitigation
dataset_pred.labels = y_pred
print_approval_rates(dataset_pred, "After Mitigation")

metric_adv = ClassificationMetric(
    dataset_orig_test, dataset_pred,
    privileged_groups=[{"Gender": 1}], unprivileged_groups=[{"Gender": 0}]
)

print("\nFairness Metrics After Mitigation:")
print(f" Accuracy: {accuracy_score(dataset_orig_test.labels, y_pred):.4f}")
print(f" Disparate Impact (DI): {metric_adv.disparate_impact():.4f}")
print(f" Statistical Parity Difference (SPD): {metric_adv.statistical_parity_difference():.4f}")
print(f" Equal Opportunity Difference (EOD): {metric_adv.equal_opportunity_difference():.4f}")
print(f" Average Odds Difference (AOD): {metric_adv.average_odds_difference():.4f}")



Approval Rates - Original Test Data:
  Privileged Group (Male) Approval Rate: 0.8895
  Unprivileged Group (Female) Approval Rate: 0.4363

Fairness Metrics Before Mitigation:
 Accuracy: 0.7787
 Disparate Impact (DI): 0.6461
 Statistical Parity Difference (SPD): -0.3537
 Equal Opportunity Difference (EOD): -0.3257
 Average Odds Difference (AOD): -0.3508

Approval Rates - After Mitigation:
  Privileged Group (Male) Approval Rate: 0.9995
  Unprivileged Group (Female) Approval Rate: 0.9988

Fairness Metrics After Mitigation:
 Accuracy: 0.7583
 Disparate Impact (DI): 0.9993
 Statistical Parity Difference (SPD): -0.0007
 Equal Opportunity Difference (EOD): -0.0021
 Average Odds Difference (AOD): -0.0011


# LFR

In [22]:
from aif360.algorithms.preprocessing import LFR
from aif360.metrics import ClassificationMetric
from aif360.datasets import StandardDataset
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

# 1. Normalize Data (LFR requires normalized input)
scaler = StandardScaler()
dataset_orig_train.features = scaler.fit_transform(dataset_orig_train.features)
dataset_orig_test.features = scaler.transform(dataset_orig_test.features)

# 2. Compute Fairness Metrics Before Mitigation
print("Approval Rates - Original Test Data:")
print_approval_rates(dataset_orig_test, "Original Test Data")

dataset_pred = dataset_orig_test.copy()
dataset_pred.labels = model.predict(dataset_orig_test.features)  # Use baseline model predictions

metric_test = ClassificationMetric(
    dataset_orig_test, dataset_pred,
    privileged_groups=[{"Gender": 1}],
    unprivileged_groups=[{"Gender": 0}]
)

print("Fairness Metrics Before Mitigation:")
print(f"  Accuracy: {accuracy_score(dataset_orig_test.labels, dataset_pred.labels):.4f}")
print(f"  Disparate Impact (DI): {metric_test.disparate_impact():.4f}")
print(f"  Statistical Parity Difference (SPD): {metric_test.statistical_parity_difference():.4f}")
print(f"  Equal Opportunity Difference (EOD): {metric_test.equal_opportunity_difference():.4f}")
print(f"  Average Odds Difference (AOD): {metric_test.average_odds_difference():.4f}")

# 3. Apply LFR Bias Mitigation
print("\nApplying LFR Bias Mitigation...")
lfr = LFR(
    privileged_groups=[{"Gender": 1}],
    unprivileged_groups=[{"Gender": 0}],
    k=5,                # Number of clusters
    Ax=0.01,            # Accuracy weight
    Ay=1.0,             # Fairness weight
    Az=0.1,             # Adversary weight
    verbose=1
)

lfr.fit(dataset_orig_train)
dataset_train_transf = lfr.transform(dataset_orig_train)
dataset_test_transf = lfr.transform(dataset_orig_test)

# 4. Train Logistic Regression on Transformed Data
print("\nTraining Logistic Regression on Transformed Data...")
model_lfr = LogisticRegression(solver='liblinear')
model_lfr.fit(dataset_train_transf.features, dataset_orig_train.labels.ravel())
y_pred_lfr = model_lfr.predict(dataset_test_transf.features)

# `dataset_pred_lfr` keeps original test features
dataset_pred_lfr = dataset_orig_test.copy()
dataset_pred_lfr.labels = y_pred_lfr

# 5. Compute Fairness Metrics After LFR Mitigation
print("\nApproval Rates - After LFR Mitigation:")
print_approval_rates(dataset_pred_lfr, "After LFR Mitigation")

metric_lfr = ClassificationMetric(
    dataset_orig_test, dataset_pred_lfr,
    privileged_groups=[{"Gender": 1}],
    unprivileged_groups=[{"Gender": 0}]
)

print("Fairness Metrics After LFR Mitigation:")
print(f"  Accuracy: {accuracy_score(dataset_orig_test.labels, y_pred_lfr):.4f}")
print(f"  Disparate Impact (DI): {metric_lfr.disparate_impact():.4f}")
print(f"  Statistical Parity Difference (SPD): {metric_lfr.statistical_parity_difference():.4f}")
print(f"  Equal Opportunity Difference (EOD): {metric_lfr.equal_opportunity_difference():.4f}")
print(f"  Average Odds Difference (AOD): {metric_lfr.average_odds_difference():.4f}")


Approval Rates - Original Test Data:

Approval Rates - Original Test Data:
  Privileged Group (Male) Approval Rate: 0.8830
  Unprivileged Group (Female) Approval Rate: 0.4421
Fairness Metrics Before Mitigation:
  Accuracy: 0.6527
  Disparate Impact (DI): 0.0983
  Statistical Parity Difference (SPD): -0.6155
  Equal Opportunity Difference (EOD): -0.6168
  Average Odds Difference (AOD): -0.5285

Applying LFR Bias Mitigation...
step: 0, loss: 0.7501681277363016, L_x: 2.735386100706961,  L_y: 0.7206571060548127,  L_z: 0.021571606744192617
step: 250, loss: 0.5874261308404015, L_x: 2.736522071094791,  L_y: 0.5587051861623932,  L_z: 0.013557239670604165
step: 500, loss: 0.5204346087212252, L_x: 2.400399487776183,  L_y: 0.4820864862565963,  L_z: 0.14344127586867067
step: 750, loss: 0.4836706958238537, L_x: 2.273497971713118,  L_y: 0.44344797706564754,  L_z: 0.17487739041074998
step: 1000, loss: 0.4702915660686823, L_x: 2.2059624218020435,  L_y: 0.4292399594824674,  L_z: 0.1899198236819444
step

# New Section

In [ ]:
from aif360.algorithms.preprocessing import DisparateImpactRemover
from aif360.metrics import ClassificationMetric
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Define privileged and unprivileged groups
privileged_groups = [{'Gender': 1}]
unprivileged_groups = [{'Gender': 0}]

# Apply Disparate Impact Remover (set repair_level between 0 to 1)
dir = DisparateImpactRemover(repair_level=0.8)
dataset_dir_train = dir.fit_transform(dataset_orig_train)

# Train logistic regression model on transformed data
X_train = dataset_dir_train.features
y_train = dataset_dir_train.labels.ravel()

model = LogisticRegression(solver='liblinear')
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(dataset_orig_test.features)

# Evaluate Fairness Metrics After DIR
dataset_pred = dataset_orig_test.copy()
dataset_pred.labels = y_pred

metric_dir = ClassificationMetric(dataset_orig_test, dataset_pred,
                                  privileged_groups=privileged_groups,
                                  unprivileged_groups=unprivileged_groups)

print("\n Fairness Metrics After Disparate Impact Remover:")
print(f" Accuracy: {accuracy_score(dataset_orig_test.labels, y_pred):.4f}")
print(f" Disparate Impact (DI): {metric_dir.disparate_impact():.4f}")
print(f" Statistical Parity Difference (SPD): {metric_dir.statistical_parity_difference():.4f}")
print(f" Equal Opportunity Difference (EOD): {metric_dir.equal_opportunity_difference():.4f}")
print(f" Average Odds Difference (AOD): {metric_dir.average_odds_difference():.4f}")



 Fairness Metrics After Disparate Impact Remover:
 Accuracy: 0.8160
 Disparate Impact (DI): 0.5975
 Statistical Parity Difference (SPD): -0.4021
 Equal Opportunity Difference (EOD): -0.2087
 Average Odds Difference (AOD): -0.3786


In [ ]:
# Import necessary libraries
from aif360.algorithms.preprocessing import DisparateImpactRemover, Reweighing
from aif360.metrics import ClassificationMetric
from aif360.datasets import StandardDataset
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Define privileged and unprivileged groups
privileged_groups = [{'Gender': 1}]  # Assuming 1 represents privileged (Male)
unprivileged_groups = [{'Gender': 0}]  # Assuming 0 represents unprivileged (Female)

# Step 1: Apply Disparate Impact Remover (DIR)
dir = DisparateImpactRemover(repair_level=0.8)  # You can adjust repair_level (0.1 to 1.0)
dataset_dir = dir.fit_transform(dataset_orig)

# Step 2: Apply Reweighing
reweigh = Reweighing(privileged_groups=privileged_groups, unprivileged_groups=unprivileged_groups)
dataset_reweighed = reweigh.fit_transform(dataset_dir)

# Step 3: Train a Classifier (Logistic Regression)
X_train = dataset_reweighed.features
y_train = dataset_reweighed.labels.ravel()
sample_weights = dataset_reweighed.instance_weights

model = LogisticRegression(solver='liblinear')
model.fit(X_train, y_train, sample_weight=sample_weights)

# Step 4: Evaluate Model on Original Test Set
y_pred = model.predict(dataset_orig_test.features)
dataset_pred = dataset_orig_test.copy()
dataset_pred.labels = y_pred

# Compute Fairness Metrics
metric_combined = ClassificationMetric(dataset_orig_test, dataset_pred,
                                       privileged_groups=privileged_groups,
                                       unprivileged_groups=unprivileged_groups)

# Print Fairness Metrics
print("\n Fairness Metrics After Combining DIR + Reweighing:")
print(f" Accuracy: {accuracy_score(dataset_orig_test.labels, y_pred):.4f}")
print(f" Disparate Impact (DI): {metric_combined.disparate_impact():.4f}")
print(f" Statistical Parity Difference (SPD): {metric_combined.statistical_parity_difference():.4f}")
print(f" Equal Opportunity Difference (EOD): {metric_combined.equal_opportunity_difference():.4f}")
print(f" Average Odds Difference (AOD): {metric_combined.average_odds_difference():.4f}")



 Fairness Metrics After Combining DIR + Reweighing:
 Accuracy: 0.7727
 Disparate Impact (DI): 0.9184
 Statistical Parity Difference (SPD): -0.0752
 Equal Opportunity Difference (EOD): -0.0107
 Average Odds Difference (AOD): 0.0187


# Adversial debiassing

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score
from aif360.algorithms.inprocessing import AdversarialDebiasing
from aif360.datasets import StandardDataset
from aif360.metrics import ClassificationMetric
import tensorflow as tf
tf.compat.v1.disable_eager_execution()
# Load dataset
df = pd.read_csv("/content/biased_loan_dataset.csv")

# Fill missing values
df.fillna(df.mode().iloc[0], inplace=True)

# Drop Loan_ID if it exists
if "Loan_ID" in df.columns:
    df.drop(columns=["Loan_ID"], inplace=True)

# Encode categorical variables
gender_map = {'Male': 1, 'Female': 0}
loan_status_map = {'Y': 1, 'N': 0}

df["Gender"] = df["Gender"].map(gender_map)
df["Loan_Status"] = df["Loan_Status"].map(loan_status_map)

# Convert categorical features to numerical
categorical_features = ['Married', 'Dependents', 'Education', 'Self_Employed', 'Property_Area']
for feature in categorical_features:
    if feature in df.columns:
        df[feature] = df[feature].astype("category").cat.codes

# Scale numeric features
scaler = MinMaxScaler()
num_cols = df.select_dtypes(include=["int64", "float64"]).columns
df[num_cols] = scaler.fit_transform(df[num_cols])

# Convert dataset to AIF360 format
dataset_orig = StandardDataset(
    df,
    label_name="Loan_Status",
    favorable_classes=[1],
    protected_attribute_names=["Gender"],
    privileged_classes=[[1]]
)

# Split into Train & Test
dataset_orig_train, dataset_orig_test = dataset_orig.split([0.7], shuffle=True)

# Reset TensorFlow Session
tf.compat.v1.reset_default_graph()
sess = tf.compat.v1.Session()

# Train Adversarial Debiasing Model
adv_debiasing = AdversarialDebiasing(
    privileged_groups=[{"Gender": 1}],
    unprivileged_groups=[{"Gender": 0}],
    scope_name='adv_debiasing',
    sess=sess,
    num_epochs=50,
    batch_size=64,
    adversary_loss_weight=0.8
)

adv_debiasing.fit(dataset_orig_train)

# Predict
dataset_debiased_test = adv_debiasing.predict(dataset_orig_test)
y_pred_adv = dataset_debiased_test.labels

# Debug: Check Predictions
print("Unique values in y_pred_adv:", np.unique(y_pred_adv))

# Fairness Metrics
metric_adv = ClassificationMetric(
    dataset_orig_test, dataset_debiased_test,
    privileged_groups=[{"Gender": 1}], unprivileged_groups=[{"Gender": 0}]
)

# Handle NaN values safely
di_value = metric_adv.disparate_impact()
if np.isnan(di_value) or di_value == np.inf:
    di_value = "Undefined (Possible Zero Division)"

spd_value = metric_adv.statistical_parity_difference()
if np.isnan(spd_value):
    spd_value = "Undefined (All predictions may be the same)"

eod_value = metric_adv.equal_opportunity_difference()
if np.isnan(eod_value):
    eod_value = "Undefined (Check predictions distribution)"

aod_value = metric_adv.average_odds_difference()
if np.isnan(aod_value):
    aod_value = "Undefined (Check class predictions)"

# Print Fairness Metrics
print("\nFairness Metrics After Adversarial Debiasing:")
print(f" Accuracy: {accuracy_score(dataset_orig_test.labels, dataset_debiased_test.labels):.4f}")
print(f" Disparate Impact (DI): {di_value}")
print(f" Statistical Parity Difference (SPD): {spd_value}")
print(f" Equal Opportunity Difference (EOD): {eod_value}")
print(f" Average Odds Difference (AOD): {aod_value}")

# Close TensorFlow session
sess.close()


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


epoch 0; iter: 0; batch classifier loss: 0.791546; batch adversarial loss: 0.777074
epoch 1; iter: 0; batch classifier loss: 0.529304; batch adversarial loss: 0.891982
epoch 2; iter: 0; batch classifier loss: 0.395589; batch adversarial loss: 0.820451
epoch 3; iter: 0; batch classifier loss: 0.569855; batch adversarial loss: 0.715205
epoch 4; iter: 0; batch classifier loss: 0.376829; batch adversarial loss: 0.687002
epoch 5; iter: 0; batch classifier loss: 0.425736; batch adversarial loss: 0.623801
epoch 6; iter: 0; batch classifier loss: 0.397586; batch adversarial loss: 0.577656
epoch 7; iter: 0; batch classifier loss: 0.449312; batch adversarial loss: 0.584108
epoch 8; iter: 0; batch classifier loss: 0.355115; batch adversarial loss: 0.583136
epoch 9; iter: 0; batch classifier loss: 0.442670; batch adversarial loss: 0.517555
epoch 10; iter: 0; batch classifier loss: 0.244683; batch adversarial loss: 0.451203
epoch 11; iter: 0; batch classifier loss: 0.412883; batch adversarial loss:

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score
from aif360.algorithms.inprocessing import AdversarialDebiasing
from aif360.datasets import StandardDataset
from aif360.metrics import ClassificationMetric
import tensorflow as tf
tf.compat.v1.disable_eager_execution()
def print_approval_rates(dataset, label):
    privileged = dataset.labels[dataset.protected_attributes[:, 0] == 1]
    unprivileged = dataset.labels[dataset.protected_attributes[:, 0] == 0]

    priv_rate = np.mean(privileged)
    unpriv_rate = np.mean(unprivileged)

    print(f"\nApproval Rates - {label}:")
    print(f"  Privileged Group (Male) Approval Rate: {priv_rate:.4f}")
    print(f"  Unprivileged Group (Female) Approval Rate: {unpriv_rate:.4f}")

# Load dataset
df = pd.read_csv("/content/biased_loan_dataset.csv")

# Fill missing values
df.fillna(df.mode().iloc[0], inplace=True)

# Drop Loan_ID if it exists
if "Loan_ID" in df.columns:
    df.drop(columns=["Loan_ID"], inplace=True)

# Encode categorical variables
gender_map = {'Male': 1, 'Female': 0}
loan_status_map = {'Y': 1, 'N': 0}

df["Gender"] = df["Gender"].map(gender_map)
df["Loan_Status"] = df["Loan_Status"].map(loan_status_map)

# Convert categorical features to numerical
categorical_features = ['Married', 'Dependents', 'Education', 'Self_Employed', 'Property_Area']
for feature in categorical_features:
    if feature in df.columns:
        df[feature] = df[feature].astype("category").cat.codes

# Scale numeric features
scaler = MinMaxScaler()
num_cols = df.select_dtypes(include=["int64", "float64"]).columns
df[num_cols] = scaler.fit_transform(df[num_cols])

# Convert dataset to AIF360 format
dataset_orig = StandardDataset(
    df,
    label_name="Loan_Status",
    favorable_classes=[1],
    protected_attribute_names=["Gender"],
    privileged_classes=[[1]]
)

# Split into Train & Test
dataset_orig_train, dataset_orig_test = dataset_orig.split([0.7], shuffle=True)

# Print Approval Rates Before Mitigation
print_approval_rates(dataset_orig_test, "Original Test Data")

# Reset TensorFlow Session
tf.compat.v1.reset_default_graph()
sess = tf.compat.v1.Session()

# Train Adversarial Debiasing Model
adv_debiasing = AdversarialDebiasing(
    privileged_groups=[{"Gender": 1}],
    unprivileged_groups=[{"Gender": 0}],
    scope_name='adv_debiasing',
    sess=sess,
    num_epochs=100,
    batch_size=64,
    adversary_loss_weight=0.5
)

adv_debiasing.fit(dataset_orig_train)

# Predict
dataset_debiased_test = adv_debiasing.predict(dataset_orig_test)
y_pred_adv = dataset_debiased_test.labels

# Print Approval Rates After Mitigation
print_approval_rates(dataset_debiased_test, "After Mitigation")

# Fairness Metrics
metric_adv = ClassificationMetric(
    dataset_orig_test, dataset_debiased_test,
    privileged_groups=[{"Gender": 1}], unprivileged_groups=[{"Gender": 0}]
)

# Handle NaN values safely
di_value = metric_adv.disparate_impact()
if np.isnan(di_value) or di_value == np.inf:
    di_value = "Undefined (Possible Zero Division)"

spd_value = metric_adv.statistical_parity_difference()
if np.isnan(spd_value):
    spd_value = "Undefined (All predictions may be the same)"

eod_value = metric_adv.equal_opportunity_difference()
if np.isnan(eod_value):
    eod_value = "Undefined (Check predictions distribution)"

aod_value = metric_adv.average_odds_difference()
if np.isnan(aod_value):
    aod_value = "Undefined (Check class predictions)"

# Print Fairness Metrics
print("\nFairness Metrics After Adversarial Debiasing:")
print(f" Accuracy: {accuracy_score(dataset_orig_test.labels, dataset_debiased_test.labels):.4f}")
print(f" Disparate Impact (DI): {di_value}")
print(f" Statistical Parity Difference (SPD): {spd_value}")
print(f" Equal Opportunity Difference (EOD): {eod_value}")
print(f" Average Odds Difference (AOD): {aod_value}")

# Close TensorFlow session
sess.close()



Approval Rates - Original Test Data:
  Privileged Group (Male) Approval Rate: 0.8788
  Unprivileged Group (Female) Approval Rate: 0.4350
epoch 0; iter: 0; batch classifier loss: 0.836553; batch adversarial loss: 0.711505
epoch 1; iter: 0; batch classifier loss: 0.456551; batch adversarial loss: 0.691397
epoch 2; iter: 0; batch classifier loss: 0.394828; batch adversarial loss: 0.647422
epoch 3; iter: 0; batch classifier loss: 0.363318; batch adversarial loss: 0.590260
epoch 4; iter: 0; batch classifier loss: 0.440912; batch adversarial loss: 0.552169
epoch 5; iter: 0; batch classifier loss: 0.351597; batch adversarial loss: 0.542994
epoch 6; iter: 0; batch classifier loss: 0.457447; batch adversarial loss: 0.594860
epoch 7; iter: 0; batch classifier loss: 0.377334; batch adversarial loss: 0.497082
epoch 8; iter: 0; batch classifier loss: 0.480294; batch adversarial loss: 0.536601
epoch 9; iter: 0; batch classifier loss: 0.348584; batch adversarial loss: 0.499285
epoch 10; iter: 0; bat

# New Section

In [24]:
from aif360.algorithms.inprocessing import PrejudiceRemover

# Initialize and train Prejudice Remover
prejudice_remover = PrejudiceRemover(eta=1.0)  # 'eta' controls fairness-accuracy tradeoff
prejudice_remover.fit(dataset_orig_train)

# Predict on test data
dataset_pred = prejudice_remover.predict(dataset_orig_test)

# Compute fairness metrics
metric_pr = ClassificationMetric(dataset_orig_test, dataset_pred,
                                 privileged_groups=[{"Gender": 1}],
                                 unprivileged_groups=[{"Gender": 0}])

print("\nFairness Metrics After Prejudice Remover:")
print(f" Accuracy: {accuracy_score(dataset_orig_test.labels.ravel(), dataset_pred.labels.ravel()):.4f}")
print(f" Disparate Impact (DI): {metric_pr.disparate_impact():.4f}")
print(f" Statistical Parity Difference (SPD): {metric_pr.statistical_parity_difference():.4f}")
print(f" Equal Opportunity Difference (EOD): {metric_pr.equal_opportunity_difference():.4f}")
print(f" Average Odds Difference (AOD): {metric_pr.average_odds_difference():.4f}")



Fairness Metrics After Prejudice Remover:
 Accuracy: 0.8063
 Disparate Impact (DI): 0.7072
 Statistical Parity Difference (SPD): -0.2832
 Equal Opportunity Difference (EOD): -0.1411
 Average Odds Difference (AOD): -0.2205


In [38]:
# Import necessary libraries
from aif360.algorithms.inprocessing import PrejudiceRemover
from aif360.metrics import ClassificationMetric
from sklearn.metrics import accuracy_score

# Function to Calculate Overall Acceptance Rate
def calculate_acceptance_rate(dataset):
    """Calculates the percentage of accepted applications (label = 1)."""
    return (dataset.labels.ravel().sum() / len(dataset.labels)) * 100

# Function to Print Approval Rates for Privileged & Unprivileged Groups
def print_approval_rates(dataset, label):
    """Prints the approval rate for privileged and unprivileged groups."""
    privileged_filter = dataset.protected_attributes[:, 0] == 1  # Gender=1 (Male)
    unprivileged_filter = dataset.protected_attributes[:, 0] == 0  # Gender=0 (Female)

    privileged_approval_rate = dataset.labels[privileged_filter].mean()
    unprivileged_approval_rate = dataset.labels[unprivileged_filter].mean()

    print(f"\nApproval Rates - {label}:")
    print(f"  Privileged Group (Male) Approval Rate: {privileged_approval_rate:.4f}")
    print(f"  Unprivileged Group (Female) Approval Rate: {unprivileged_approval_rate:.4f}")

# Compute Approval Rates Before Mitigation
print_approval_rates(dataset_orig_test, "Before Mitigation")

# Compute Overall Acceptance Rate Before Mitigation
orig_acceptance_rate = calculate_acceptance_rate(dataset_orig_test)
print(f"\nOverall Acceptance Rate Before Mitigation: {orig_acceptance_rate:.2f}%")

# Apply Prejudice Remover
print("\nApplying Prejudice Remover...")
prejudice_remover = PrejudiceRemover(eta=1.0)  # 'eta' controls fairness-accuracy tradeoff
prejudice_remover.fit(dataset_orig_train)

# Predict on Test Data
dataset_pred = prejudice_remover.predict(dataset_orig_test)

# Compute Approval Rates After Mitigation
print_approval_rates(dataset_pred, "After Mitigation")

# Compute Overall Acceptance Rate After Mitigation
mitigated_acceptance_rate = calculate_acceptance_rate(dataset_pred)
print(f"\nOverall Acceptance Rate After Mitigation: {mitigated_acceptance_rate:.2f}%")

# Compute Fairness Metrics
metric_pr = ClassificationMetric(dataset_orig_test, dataset_pred,
                                 privileged_groups=[{"Gender": 1}],
                                 unprivileged_groups=[{"Gender": 0}])

# Display Fairness Metrics
print("\nFairness Metrics After Prejudice Remover:")
print(f" Accuracy: {accuracy_score(dataset_orig_test.labels.ravel(), dataset_pred.labels.ravel()):.4f}")
print(f" Disparate Impact (DI): {metric_pr.disparate_impact():.4f}")
print(f" Statistical Parity Difference (SPD): {metric_pr.statistical_parity_difference():.4f}")
print(f" Equal Opportunity Difference (EOD): {metric_pr.equal_opportunity_difference():.4f}")
print(f" Average Odds Difference (AOD): {metric_pr.average_odds_difference():.4f}")



Approval Rates - Before Mitigation:
  Privileged Group (Male) Approval Rate: 0.8830
  Unprivileged Group (Female) Approval Rate: 0.4421

Overall Acceptance Rate Before Mitigation: 75.60%

Applying Prejudice Remover...

Approval Rates - After Mitigation:
  Privileged Group (Male) Approval Rate: 0.9672
  Unprivileged Group (Female) Approval Rate: 0.6840

Overall Acceptance Rate After Mitigation: 88.57%

Fairness Metrics After Prejudice Remover:
 Accuracy: 0.8063
 Disparate Impact (DI): 0.7072
 Statistical Parity Difference (SPD): -0.2832
 Equal Opportunity Difference (EOD): -0.1411
 Average Odds Difference (AOD): -0.2205


In [29]:
from aif360.algorithms.postprocessing import EqOddsPostprocessing

eq_odds = EqOddsPostprocessing(privileged_groups=[{"Gender": 1}],
                               unprivileged_groups=[{"Gender": 0}])
dataset_eq_odds = eq_odds.fit_predict(dataset_orig_test, dataset_pred)

metric_eq_odds = ClassificationMetric(dataset_orig_test, dataset_eq_odds,
                                      privileged_groups=[{"Gender": 1}],
                                      unprivileged_groups=[{"Gender": 0}])

print("\n Fairness Metrics After Equalized Odds Post-processing:")
print(f" Accuracy: {accuracy_score(y_test, dataset_eq_odds.labels):.4f}")
print(f" Disparate Impact (DI): {metric_eq_odds.disparate_impact():.4f}")
print(f" Statistical Parity Difference (SPD): {metric_eq_odds.statistical_parity_difference():.4f}")
print(f" Equal Opportunity Difference (EOD): {metric_eq_odds.equal_opportunity_difference():.4f}")
print(f" Average Odds Difference (AOD): {metric_eq_odds.average_odds_difference():.4f}")




 Fairness Metrics After Equalized Odds Post-processing:
 Accuracy: 0.7493
 Disparate Impact (DI): 0.9439
 Statistical Parity Difference (SPD): -0.0517
 Equal Opportunity Difference (EOD): 0.0140
 Average Odds Difference (AOD): -0.0105


In [30]:
from aif360.algorithms.postprocessing import RejectOptionClassification

roc = RejectOptionClassification(privileged_groups=[{"Gender": 1}],
                                 unprivileged_groups=[{"Gender": 0}],
                                 low_class_thresh=0.01,
                                 high_class_thresh=0.99,
                                 num_class_thresh=100,
                                 metric_name="Statistical parity difference",
                                 metric_ub=0.05, metric_lb=-0.05)

dataset_roc = roc.fit_predict(dataset_orig_test, dataset_pred)

metric_roc = ClassificationMetric(dataset_orig_test, dataset_roc,
                                  privileged_groups=[{"Gender": 1}],
                                  unprivileged_groups=[{"Gender": 0}])

print("\n Fairness Metrics After Reject Option Classification:")
print(f" Accuracy: {accuracy_score(y_test, dataset_roc.labels):.4f}")
print(f" Disparate Impact (DI): {metric_roc.disparate_impact():.4f}")
print(f" Statistical Parity Difference (SPD): {metric_roc.statistical_parity_difference():.4f}")
print(f" Equal Opportunity Difference (EOD): {metric_roc.equal_opportunity_difference():.4f}")
print(f" Average Odds Difference (AOD): {metric_roc.average_odds_difference():.4f}")



 Fairness Metrics After Reject Option Classification:
 Accuracy: 0.7270
 Disparate Impact (DI): 0.9402
 Statistical Parity Difference (SPD): -0.0446
 Equal Opportunity Difference (EOD): 0.0612
 Average Odds Difference (AOD): 0.1271


# Implementaion with fairlearn